Python中的协程大概经历了如下三个阶段：
1. 最初的生成器变形yield/send
2. 引入@asyncio.coroutine和yield from
3. 在最近的Python3.5版本中引入async/await关键字

### 一 生成器变形yield/send

普通函数中如果出现了yield关键字，那么该函数就不再是普通函数，而是一个生成器。

In [2]:
def mygen(alist):
    while len(alist) > 0:
        c = randint(0, len(alist)-1)
        yield alist.pop(c)

a = ["aa","bb","cc"]

In [4]:
c = mygen(a)
print(c)

<generator object mygen at 0x7fb7e83bc890>


像上面代码中的c就是一个生成器。生成器就是一种迭代器，可以使用for进行迭代。生成器函数最大的特点是可以接受外部传入的一个变量，并根据变量内容计算结果后返回。
这一切都是靠生成器内部的send()函数实现的。

In [5]:
def gen():
    value=0
    while True:
        receive=yield value
        if receive=='e':
            break
        value = 'got: %s' % receive

g=gen()
print(g.send(None))    
print(g.send('hello'))
print(g.send(123456))
print(g.send('e'))

0
got: hello
got: 123456


StopIteration: 

上面生成器函数中最关键也是最易理解错的，就是receive=yield value这句,如果对循环体的执行步骤理解错误，就会失之毫厘，差之千里。

其实receive=yield value包含了3个步骤：
1. 向函数外抛出（返回）value
2. 暂停(pause)，等待next()或send()恢复
3. 赋值receive=MockGetValue() 。 这个MockGetValue()是假想函数，用来接收send()发送进来的值

执行流程：
1. 通过`g.send(None)` 或者`next(g)` 启动生成器函数，并执行到第一个 yield 语句结束的位置。实际程序只执行了1，2两步，程序返回了 value 值，并暂停(pause)，并没有执行第3步给receive赋值。因此`yield value`会输出初始值0。这里要特别注意：在启动生成器函数时只能`send(None)`,如果试图输入其它的值都会得到错误提示信息。

2. 通过`g.send('hello')`，会传入hello，从上次暂停的位置继续执行，那么就是运行第3步，赋值给receive。然后计算出value的值，并回到while头部，遇到yield value，程序再次执行了1，2两步，程序返回了value值，并暂停(pause)。此时yield value会输出”got: hello”，并等待send()激活。

3. 通过g.send(123456)，会重复第2步，最后输出结果为”got: 123456″。

4. 执行g.send(‘e’)时，程序会执行break然后推出循环，最后整个函数执行完毕，所以会得到StopIteration异常。

从上面可以看出， 在第一次send(None)启动生成器（执行1–>2，通常第一次返回的值没有什么用）之后，对于外部的每一次send()，生成器的实际在循环中的运行顺序是3–>1–>2，也就是先获取值，然后dosomething，然后返回一个值，再暂停等待。


### 二 yield from

In [6]:
def g1():     
     yield  range(5)
        
def g2():
     yield  from range(5)

In [9]:
it1 = g1()
it2 = g2()
for x in it1:
    print(x)

print("---")

for x in it2:
    print(x)

range(0, 5)
---
0
1
2
3
4


yield将range这个可迭代对象直接返回了, 而yield from解析了range对象，将其中每一个item返回

`yield from iterable` 本质上等于`for item in iterable: yield item` 的缩写版

In [15]:
def fib(max_n):
    n, a, b = 0, 0, 1
    while n < max_n:
        yield b
        a, b = b, a + b
        n = n + 1

In [16]:
def f_wrapper(fun_iterable):
    print('start')
    for item in fun_iterable:
        yield item
    print('end')


wrap = f_wrapper(fib(5))
for i in wrap:
    print(i, end=' ')

start
1 1 2 3 5 end


In [17]:
def f_wrapper2(fun_iterable):
    print('start')
    yield from fun_iterable  # 注意此处必须是一个可生成对象
    print('end')


wrap = f_wrapper2(fib(5))
for i in wrap:
    print(i, end=' ')

start
1 1 2 3 5 end


`yield from` 后面必须跟`iterable`对象(可以是生成器，迭代器)

### 三 asyncio.coroutine和yield from

yield from在asyncio模块中得以发扬光大。之前都是我们手工切换协程，现在当声明函数为协程后，我们通过事件循环来调度协程

先看示例代码：

In [ ]:
import asyncio, random


@asyncio.coroutine
def smart_fib(n):
    index = 0
    a = 0
    b = 1
    while index < n:
        sleep_secs = random.uniform(0, 0.2)
        yield from asyncio.sleep(sleep_secs)  # 通常yield from后都是接的耗时操作
        print('Smart one think {} secs to get {}'.format(sleep_secs, b))
        a, b = b, a + b
        index += 1


@asyncio.coroutine
def stupid_fib(n):
    index = 0
    a = 0
    b = 1
    while index < n:
        sleep_secs = random.uniform(0, 0.4)
        yield from asyncio.sleep(sleep_secs)  # 通常yield from后都是接的耗时操作
        print('Stupid one think {} secs to get {}'.format(sleep_secs, b))
        a, b = b, a + b
        index += 1


loop = asyncio.get_event_loop()
tasks = [smart_fib(10), stupid_fib(10)]
loop.run_until_complete(asyncio.wait(tasks))
print('All fib finished.')
loop.close()

output:
```
Smart one think 0.026525109235243163 secs to get 1
Smart one think 0.027785034767742744 secs to get 1
Stupid one think 0.13037907715928085 secs to get 1
Smart one think 0.11669741028096901 secs to get 2
Stupid one think 0.04238196174869327 secs to get 1
Stupid one think 0.11380284364243956 secs to get 2
Smart one think 0.11769596813426297 secs to get 3
Smart one think 0.17488528000457104 secs to get 5
Smart one think 0.04971763086780401 secs to get 8
Smart one think 0.05628655306689132 secs to get 13
Stupid one think 0.38969300185314415 secs to get 3
Smart one think 0.13572058551224658 secs to get 21
Smart one think 0.10732171780502052 secs to get 34
Stupid one think 0.18299519880414508 secs to get 5
Stupid one think 0.09949915850652391 secs to get 8
Smart one think 0.1512613986844706 secs to get 55
Stupid one think 0.3493415169530308 secs to get 13
Stupid one think 0.30437060390884557 secs to get 21
Stupid one think 0.1070324128128287 secs to get 34
Stupid one think 0.17135892434390954 secs to get 55
All fib finished.
```

yield from语法可以让我们方便地调用另一个generator。
本例中yield from后面接的asyncio.sleep()是一个coroutine(里面也用了yield from)，所以线程不会等待asyncio.sleep()，而是直接中断并执行下一个消息循环。当asyncio.sleep()返回时，线程就可以从yield from拿到返回值（此处是None），然后接着执行下一行语句。
asyncio是一个基于事件循环的实现异步I/O的模块。通过yield from，我们可以将协程asyncio.sleep的控制权交给事件循环，然后挂起当前协程；之后，由事件循环决定何时唤醒asyncio.sleep,接着向后执行代码。
协程之间的调度都是由事件循环决定。
yield from asyncio.sleep(sleep_secs) 这里不能用time.sleep(1)因为time.sleep()返回的是None，它不是iterable，还记得前面说的yield from后面必须跟iterable对象(可以是生成器，迭代器)。
所以会报错：
> yield from time.sleep(sleep_secs) TypeError: ‘NoneType’ object is not iterable

### 四 async和await

弄清楚了asyncio.coroutine和yield from之后，在Python3.5中引入的async和await就不难理解了：可以将他们理解成asyncio.coroutine/yield from的完美替身。当然，从Python设计的角度来说，async/await让协程表面上独立于生成器而存在，将细节都隐藏于asyncio模块之下，语法更清晰明了。
加入新的关键字 async ，可以将任何一个普通函数变成协程

In [28]:
import random
import asyncio


async def mygen(alist):
    while len(alist) > 0:
        c = random.randint(0, len(alist) - 1)
        print(alist.pop(c))


a = ["aa", "bb", "cc"]
c = mygen(a)
print(c)

<coroutine object mygen at 0x7fb7c4b37640>


在上面程序中，我们在前面加上async，该函数就变成一个协程了。

但是async对生成器是无效的。async无法将一个生成器转换成协程。
还是刚才那段代码，我们把print改成yield

In [29]:
async def mygen(alist):
    while len(alist) > 0:
        c = random.randint(0, len(alist) - 1)
        yield alist.pop(c)


a = ["aa", "bb", "cc"]
c = mygen(a)
print(c)

<async_generator object mygen at 0x7fb7f83638b0>


/tmp/ipykernel_398983/3512565305.py:8: RuntimeWarning: coroutine 'mygen' was never awaited
  c = mygen(a)


可以看到输出

<async_generator object mygen at 0x02AA7170>
并不是coroutine 协程对象

所以我们的协程代码应该是这样的,用事件循环运行协程

In [ ]:
async def mygen(alist):
    while len(alist) > 0:
        c = random.randint(0, len(alist) - 1)
        print(alist.pop(c))
        await asyncio.sleep(1)


async def main():
    strlist = ["ss", "dd", "gg"]
    intlist = [1, 2, 5, 6]
    await asyncio.gather(
        mygen(strlist), mygen(intlist)
    )
    print('All fib finished.')


asyncio.run(main())

output:
```
<coroutine object mygen at 0x7f870db4f3c0>
dd
1
gg
6
ss
5
2
All fib finished.
```